In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import cloudpickle
import base64
import pickle
import boto3
import json
import numpy as np
from datetime import datetime
from functools import partial

### invocation wrapper

In [15]:
def invoke_with_lambda(
        func,
        lambda_client = boto3.client('lambda'),
    ):
    lambda_pickled = cloudpickle.dumps(func)
    lambda_base64 = base64.encodebytes(lambda_pickled)
    lambda_str = lambda_base64.decode('utf-8')
    lambda_json = json.dumps(lambda_str)
    response = lambda_client.invoke(
          FunctionName='sprite',
          Payload=lambda_json,
    )
    response_bytes = response['Payload'].read()
    response_str = response_bytes.decode('utf-8')
    response_json = json.loads(response_str)
    result_bytes = response_json.encode('utf8')
    result_pkl = base64.decodebytes(result_bytes)
    return pickle.loads(result_pkl)

### call some functions remotely

In [26]:
hello_print = lambda o: f'this executes {o}'

In [27]:
hello_print('locally')

'this executes locally'

In [29]:
invoke_with_lambda(partial(hello_print, 'remotely'))

'this executes remotely'

In [36]:
a = np.arange(10)
def squareit():
    return a**a

In [37]:
squareit()

array([        1,         1,         4,        27,       256,      3125,
           46656,    823543,  16777216, 387420489])

In [38]:
%time invoke_with_lambda(squareit)

CPU times: user 3.16 ms, sys: 770 µs, total: 3.93 ms
Wall time: 60.5 ms


array([        1,         1,         4,        27,       256,      3125,
           46656,    823543,  16777216, 387420489])

In [39]:
%timeit invoke_with_lambda(squareit)

33.5 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
